### Experimenting AI-THOR dataset and API

In [ ]:
from ai2thor.controller import Controller
controller = Controller()
event = controller.step('MoveAhead')

: 

In [ ]:
import os
import uuid
import numpy as np
from PIL import Image
import ai2thor.controller


# Output directories
OUTPUT_DIR = "../../data/raw"
RGB_DIR = os.path.join(OUTPUT_DIR, "rgb")
DEPTH_DIR = os.path.join(OUTPUT_DIR, "depth")
INST_DIR = os.path.join(OUTPUT_DIR, "instance")
os.makedirs(RGB_DIR, exist_ok=True)
os.makedirs(DEPTH_DIR, exist_ok=True)
os.makedirs(INST_DIR, exist_ok=True)

# Initialize AI2-THOR controller with only one specific scene
controller = ai2thor.controller.Controller(
    scene="FloorPlan1",  # Only load one scene to avoid bulk downloads
    agentMode="default",
    visibilityDistance=1.5,
    gridSize=0.25,
    rotateStepDegrees=90,
    width=300,
    height=300,
    fieldOfView=90,
    renderDepthImage=True,
    renderInstanceSegmentation=True
)

# Define simple movement actions
actions = [
    {"action": "MoveAhead"},
    {"action": "RotateRight"},
    {"action": "MoveAhead"},
    {"action": "RotateLeft"},
    {"action": "MoveAhead"}
]

# Perform actions and collect visual data
for step, act in enumerate(actions):
    event = controller.step(act)

    # Save RGB image
    rgb = Image.fromarray(event.frame)
    rgb.save(os.path.join(RGB_DIR, f"rgb_{step}.png"))

    # Save Depth image (normalized for viewing)
    depth = event.depth_frame
    depth_normalized = (depth / np.max(depth) * 255).astype(np.uint8)
    Image.fromarray(depth_normalized).save(os.path.join(DEPTH_DIR, f"depth_{step}.png"))

    # Save instance segmentation frame
    inst = event.instance_segmentation_frame
    Image.fromarray(inst).save(os.path.join(INST_DIR, f"inst_{step}.png"))

    print(f"[{step}] Captured RGB, Depth, and Instance images.")

controller.stop()
print("✅ Finished data collection from FloorPlan1.")

In [ ]:
# code to convert to point-clouds
import numpy as np
import open3d as o3d

def depth_to_point_cloud(depth, rgb, fov=90):
    """
    Converts a depth map and RGB image to a point cloud using pinhole camera model.
    Assumes depth is in meters and RGB is (H, W, 3).
    """
    h, w = depth.shape
    fx = fy = 0.5 * w / np.tan(0.5 * np.radians(fov))
    cx, cy = w / 2, h / 2

    # Generate pixel grid
    i, j = np.meshgrid(np.arange(w), np.arange(h), indexing='xy')
    z = depth
    x = (i - cx) * z / fx
    y = (j - cy) * z / fy

    xyz = np.stack((x, y, z), axis=-1).reshape(-1, 3)
    rgb_flat = rgb.reshape(-1, 3) / 255.0

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(rgb_flat)
    return pcd

In [ ]:
# TODO: first load the depth and rgb images and convert them to point-clouds
pcd = ...


# save point-cloud to .ply
POINT_CLOUD_DIR = os.path.join("HOME", "data", "point_clouds")
os.makedirs(POINT_CLOUD_DIR, exist_ok=True)
o3d.io.write_point_cloud("cloud.ply", pcd)

NameError: name 'os' is not defined